# Исследование надёжности заёмщиков



In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**: исходя из первых 15 строк таблицы можно найти следующие проблемы/вопросы касательно пердставленных данных в таблице:
- колонка **days_employed** имеет отрицательные вещественные значения, хотя количество дней трудового стажа не могут быть отрицательными как и вещественными. Данную колонку можно привести к целым числам и использовать модуль числа.
- колонка **education** содержит строчные и прописные значения. Необходимо привести к строчному регистру.
- колонка **family_status** Нужно проверить данные на схожесть по смыслу, выявив кникальные значения и привести их к одному значению.
- колонка **income_type** содержит значение 'компаньон'. Данное значение хорошо было бы уточнить у сотрудников банка, что подразумевалось. К счастью, для целей иследование эти данные не оказывают влияние
- колонку **total_income** можно привести к целому числу, копейки указанные после знака '.' не окажут влияния на исследование. Так информация станет нагляднее и репрезентативнее. 
- колонку **purpose** содержит различные цели, но данные цели можно в последствии сгруппировать, выделив категории, к примеру. Данную колонку можно лемматизировать, что даст возможность выделить цели кредита по категориям.
- в строке 12 содержатся про пуски в колонках **days_employed** и **total_income**. В целом данные пропуски выглядят логично, так где нет дохода от работы нет и стажа. Конечно, у человека возможно есть трудовой стаж, но возможно для банка это уже не имеет значения без дохода, поэтому стаж тоже пропущен.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Назания колонок написаны в удобном и правильном стиле. Имеют пропуски только в колонках **days_employed** и **total_income**, что пока подтвеждает замечание о наличие пропусков, описанное выше. Типы данные в целом представлены в нужном типе, однако, как было сказано ранее колонки **days_employed** и **total_income** можно перевести в целый вид числа.

##  Предобработка данных

### Обработка пропусков

In [5]:
data[data['total_income'].isna() == True].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


**Вывод**: исходя из полученных данных пропуски содержаться только в двух колонках и совпадают. Общие число пропущенных значений это подтверждает. Данные пропуски не зависят от типа занятости. Колонка **days_employed** по всей видимости содержит стаж текущей работы, поэтому при отсуствие дохода данные отсуствуют или банк просто не интересует данная информация без дохода. Данные пропуски лучше заменить на 0 для дальнейшего удобства и наглядности. С данной задачей справится метод **.fillna()** Удаление строк не подходит так как может повлиять на итоговый вывод в силу того, что значений не малый процент от всей выгрузки.

Пропуски по трудовому стажу заменим на 0, так как в этом иследованние эта информация нам не нужна. Пропуски по доходу заменим медианами, принадлежащим группам по виду занятости

In [6]:
data['days_employed'] = data['days_employed'].fillna(0)
income_median = data.groupby('income_type')['total_income'].median()
income_median

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [7]:
def replace_income(row):
    if pd.isna(row['total_income']):
        return income_median.loc[row['income_type']]   
    return row['total_income']

data['total_income'] = data.apply(replace_income, axis = 1)      
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Замена произведена. Необходима проверка данных

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропусков не обнаружено.

### Замена типа данных

Ранее было выделено два столбца, которые желательно привести к типу целого числа. Это колонки **days_employed** и **total_income**. Также **days_employed** можно избавить от отрицательных значений. В данному случае подойдет метод **astype()**. Для нахождениямодуля можно применить функцию **abs()**.

In [9]:
data['days_employed'] = abs(data['days_employed']).astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**: Данные приведены к желаемому формату. Проверим, как стала выглядить таблица.

In [10]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Таблица стала выглядеть намного дружелюбнее и презентабельнее. В колонке **days_employed** замечено странное значение в строке 4. Выходит что человек отработал 932 года! Возможно это ошибка внесенная человечеким фактором, либо имеет какая то своя логика. В данном случае требовалось бы уточнение у сотрудников банка, однако данный столбец в последующем исследовании и для ответов на поставленные задачи не потребуется.

In [11]:
data['days_employed'] =data['days_employed']/365
data['total_income'] =data['total_income']/1000

### Обработка дубликатов

Сначала выявим явные дубликаты в таблицы методом **duplicated()** и методом **sum()**

In [12]:
data.duplicated().sum()

54

Получен результат в 54 дубликата. Сложно сказать на самом деле ли это дубликаты учитывая, что есть вероятность схожих реальных данных. Однако вероятность, что доход и стаж совпадают одновременно со всеми другими параметрами крайне мала и удаление такого малого количество строк не повляиет сильно на выборку, скорее всего \то ошибки при копировании и выгрузки базы данных. Однако отложим удаление, сначала выявим еще неявные дубликаты, чтобы разом очистить таблицу от них. Как было описанное выше колонку с образованиями нужно привести к единому регистру, также семейный статус нужно проверить на уникальные значения и общие по смыслу обьединить в один вид.

In [13]:
data['education'] = data['education'].str.lower()

In [14]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В колонке все значения семейного положения уникальны. Можно не проводить замену. Теперь после приведения к строчному регистру колонки **education** проверим сколько стало теперь дубликатов и провести их удаление методом **drop_duplicates()**


In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод**: таблица избавлена от лишней информации в виде дубликатов.

### Лемматизация

В столбце с целями кредита обнаружены значения, которые схожи по смыслу. Данные значения неплохо было бы обьекдинить в последствии в одну категорию, для этого можно создать отдельный столбец. Сейчас необходимо пременить процесс лемматизации, который поможет выделить леммы, по которым можно будет выделять категории. Нужно будет импортировать специальный метод из библиотеки **pymystem3**


In [17]:
from pymystem3 import Mystem
m=Mystem()
#создаем из столбца с целями столбец содержащий отдельные слова - возможно пригодится для категоризации
def for_lemma(purpose):
    words = ' '.join(m.lemmatize(purpose))
    return words
data['lemma'] = data['purpose'].apply(for_lemma)


#теперь посчитаем количество лемм функцией Counter
from collections import Counter
text =[]
for el in data['lemma']:
    for word in el.split():
        text.append(word)        
        
Counter(text)
    
    



Counter({'покупка': 5897,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

**Вывод**: проведенние лематизации целей и подсчет количества дает возможность в дальнейшем выделить категории в зависисмости от целей. Учитывая размер выборки, среди основных целей, которые имеют значение более 1000 можно выделить следующие:
- Кредит на недвижимость и жилье
- Кредит на образование
- Кредит на автомобиль
- Кредиты на свадьбу
- Другое, иные цели имеющий меньшей вес в предствленной выборке

### Категоризация данных

Для целей исследования удобнее будет выделить категории. В поставленной задаче нас интересуют зависимость возврата кредита от определенных факторов/признаков, соответсвенно делить на категории лучше всего в рамках данных признаков. Найдем их уникальные значения и посчитаем

Исследуем колонку отвечающую за семеной положение. Значения уже проверили они годятся для анализа, категории выделять не нужно. Подсчитаем количество значений такой колонки и для колонки из таблицы должников, в которой отберем строки содержащие задолженности и получем среднее по группам

In [18]:
debt=data[data['debt'] != 0]
data_family_group = data['family_status'].value_counts()
debt_family_group = debt['family_status'].value_counts()
averige_age_group = debt_family_group/data_family_group*100
averige_age_group

женат / замужем          7.545182
гражданский брак         9.347145
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
Name: family_status, dtype: float64

Четкой зависимости нет. Наибольший процент надежности из данных категорий у вдов и вдовцов. Перейдем к следующему фактору - наличие детей. Посмотрим сразу же на разбивку среди должников и общее количество таких лиц

In [19]:
debt['children'].value_counts()

 0     1063
 1      444
 2      194
 3       27
 20       8
 4        4
-1        1
Name: children, dtype: int64

К сожалению, понять если зависимость без отнонию к общему числу нет. Также видно что в результаты попала ошибка в виде отрицательного значения. Поэтому необходимо проверить и общее число тоже

In [20]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Отрицательных значений не так много на выборку не влияют. Однако можно такое было бы уточнить у сотрудников банка, скорее всего там должны быть либо 0 либо 1, учитывая размер обоих групп, как было сказано эти значения влияния не окажут. 

In [21]:
def children_group(children):
    if children == 0:
        return 'без детей'
    return 'с детьми'
data['children'] =data['children'].apply(children_group)

In [22]:
#проверим аналогичным способом данный параметр
debt = data[data['debt'] != 0]
data_children_group = data['children'].value_counts()
debt_children_group = debt['children'].value_counts()
averige_children_group = debt_children_group/data_children_group*100
averige_children_group

без детей    7.543822
с детьми     9.208203
Name: children, dtype: float64

Ярко выраженной тенденции снова нет, однако люди без детей склонны чаще возвращать кредиты. Посмотрим на количество значений в колонке доходов.

In [23]:
data['total_income'].value_counts().sort_values()

162.799       1
336.791       1
238.868       1
79.450        1
258.179       1
           ... 
104.693       3
150.447     145
118.514     387
172.357     502
142.594    1070
Name: total_income, Length: 18608, dtype: int64

Учитывая длинну разнообразных значений, так изучить данные не получится. Необходимо выделить группы доходов, чтобы увидить есть влияние на надежность заемщиков. Пропишем функцию для этого

In [24]:
def group_income(income):
    if income> 0:
        if 0 <income<= 50:
            return 'низкий доход'
        if 50 <income<= 100:
            return 'средний доход'
        if 100 <income<= 150:
            return 'доход выше среднего'
        if 150 <income<= 200:
            return 'высокий доход'
        if 200 <income:
            return 'очень высокий доход'
    return 'пропуски'

data['group_income'] = data['total_income'].apply(group_income)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,group_income
0,с детьми,23.115068,42,высшее,0,женат / замужем,0,F,сотрудник,0,253.875,покупка жилья,покупка жилье \n,очень высокий доход
1,с детьми,11.024658,36,среднее,1,женат / замужем,0,F,сотрудник,0,112.080,приобретение автомобиля,приобретение автомобиль \n,доход выше среднего
2,без детей,15.405479,33,среднее,1,женат / замужем,0,M,сотрудник,0,145.885,покупка жилья,покупка жилье \n,доход выше среднего
3,с детьми,11.298630,32,среднее,1,женат / замужем,0,M,сотрудник,0,267.628,дополнительное образование,дополнительный образование \n,очень высокий доход
4,без детей,932.235616,53,среднее,1,гражданский брак,1,F,пенсионер,0,158.616,сыграть свадьбу,сыграть свадьба \n,высокий доход
5,без детей,2.536986,27,высшее,0,гражданский брак,1,M,компаньон,0,255.763,покупка жилья,покупка жилье \n,очень высокий доход
6,без детей,7.887671,43,высшее,0,женат / замужем,0,F,компаньон,0,240.525,операции с жильем,операция с жилье \n,очень высокий доход
7,без детей,0.416438,50,среднее,1,женат / замужем,0,M,сотрудник,0,135.823,образование,образование \n,доход выше среднего
8,с детьми,18.983562,35,высшее,0,гражданский брак,1,F,сотрудник,0,95.856,на проведение свадьбы,на проведение свадьба \n,средний доход
9,без детей,5.994521,41,среднее,1,женат / замужем,0,M,сотрудник,0,144.425,покупка жилья для семьи,покупка жилье для семья \n,доход выше среднего


In [25]:
data['group_income'].value_counts().sort_values()

низкий доход            372
средний доход          4091
высокий доход          4764
очень высокий доход    5067
доход выше среднего    7160
Name: group_income, dtype: int64

In [26]:
# теперь перезапишем дата фрейм в переменной 'debt'  посчитаем процент должников каждой группы
debt = data[data['debt'] != 0]
data_income_group = data['group_income'].value_counts()
debt_income_group = debt['group_income'].value_counts()
averige_income_group = debt_income_group/data_income_group*100
averige_income_group

высокий доход          8.501259
доход выше среднего    8.715084
низкий доход           6.182796
очень высокий доход    7.065325
средний доход          8.090931
Name: group_income, dtype: float64

Как оказалось наличие дохода у людей имеет еще меньший разброс, чем наличие детей. Наибольшей благо надежностью обладают лица с наименьшей доходной группой, возможно это связано с тем, что им дают на кредит меньшие суммы, которые легче вернуть. Но стоит учитывать, что в данной выборке их совсем мало, может поэтому их показатели лучше чем у других групп доходности.

Аналогичными методами изучим последний фактор, отражающей цели кредита. Ранее была проведена лемматизация колонки с целями и в таблицу добавлена одельнная колонка содержащяя леммы. Также были выделены категории обладающие наибольшим весом по ним пропищем функцию, которая создаст колонку с категориями.


In [27]:
def group_purpose(lemma_purpose):
    if ('жилье' in lemma_purpose or
    'недвижимость' in lemma_purpose):
        return 'кредит на недвижимость и жилье'
    elif 'свадьба' in lemma_purpose:
        return 'кредит на свадьбу'
    elif 'образование' in lemma_purpose:
        return 'кредит на образование'
    elif 'строительство' in lemma_purpose:
        return 'кредит на строительство'
    elif 'автомобиль' in lemma_purpose:
        return 'кредит на автомобиль'
    else:
        return 'Кредит на другие цели'

data['group_purpose'] = data['lemma'].apply(group_purpose)    
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,group_income,group_purpose
0,с детьми,23.115068,42,высшее,0,женат / замужем,0,F,сотрудник,0,253.875,покупка жилья,покупка жилье \n,очень высокий доход,кредит на недвижимость и жилье
1,с детьми,11.024658,36,среднее,1,женат / замужем,0,F,сотрудник,0,112.080,приобретение автомобиля,приобретение автомобиль \n,доход выше среднего,кредит на автомобиль
2,без детей,15.405479,33,среднее,1,женат / замужем,0,M,сотрудник,0,145.885,покупка жилья,покупка жилье \n,доход выше среднего,кредит на недвижимость и жилье
3,с детьми,11.298630,32,среднее,1,женат / замужем,0,M,сотрудник,0,267.628,дополнительное образование,дополнительный образование \n,очень высокий доход,кредит на образование
4,без детей,932.235616,53,среднее,1,гражданский брак,1,F,пенсионер,0,158.616,сыграть свадьбу,сыграть свадьба \n,высокий доход,кредит на свадьбу


In [28]:
#теперь посчитаем количество значений
data['group_purpose'].value_counts()

кредит на недвижимость и жилье    10811
кредит на автомобиль               4306
кредит на образование              4013
кредит на свадьбу                  2324
Name: group_purpose, dtype: int64

Найдем аналогичным способом процентное соотношение групп по целям

In [29]:
debt = data[data['debt'] != 0]
data_purpose_group = data['group_purpose'].value_counts()
debt_purpose_group = debt['group_purpose'].value_counts()
averige_purpose_group = debt_purpose_group/data_purpose_group*100
averige_purpose_group

кредит на недвижимость и жилье    7.233373
кредит на автомобиль              9.359034
кредит на образование             9.220035
кредит на свадьбу                 8.003442
Name: group_purpose, dtype: float64

Аналогично предыдущим факторам тенденции по руппам также не выявлено. Если говорить о наиболее надежной, то кредитование жилья по данной выборки обладает наибольшей надежностью. Также для наглядности можно вывести сводную таблицу, чтобы увидеть разбивку по всем факторам сразу.

In [30]:
# строим сводную таблицу по категориям
data['debt_special'] = data['debt']*100
data.pivot_table(index = ['children','group_income','group_purpose'], columns = 'family_status', values = 'debt_special', aggfunc = 'mean')

family_status                                                 Не женат / не замужем  \
children  group_income        group_purpose                                           
без детей высокий доход       кредит на автомобиль                        13.392857   
                              кредит на недвижимость и жилье               8.934708   
                              кредит на образование                       10.679612   
                              кредит на свадьбу                                 NaN   
          доход выше среднего кредит на автомобиль                        12.820513   
                              кредит на недвижимость и жилье               6.791569   
                              кредит на образование                       12.000000   
                              кредит на свадьбу                                 NaN   
          низкий доход        кредит на автомобиль                        33.333333   
                              кредит на недвижимость и жилье               0.000000   
                              кредит на образование                       20.000000   
                              кредит на свадьбу                                 NaN   
          очень высокий доход кредит на автомобиль                        11.764706   
                              кредит на недвижимость и жилье               7.643312   
                              кредит на образование                       10.434783   
                              кредит на свадьбу                                 NaN   
          средний доход       кредит на автомобиль                        10.112360   
                              кредит на недвижимость и жилье               6.751055   
                              кредит на образование                       10.714286   
                              кредит на свадьбу                                 NaN   
с детьми  высокий доход       кредит на автомобиль                        23.076923   
                              кредит на недвижимость и жилье              16.216216   
                              кредит на образование                        3.703704   
                              кредит на свадьбу                                 NaN   
          доход выше среднего кредит на автомобиль                        20.408163   
                              кредит на недвижимость и жилье               7.079646   
                              кредит на образование                       15.555556   
                              кредит на свадьбу                                 NaN   
          низкий доход        кредит на автомобиль                         0.000000   
                              кредит на недвижимость и жилье                    NaN   
                              кредит на образование                             NaN   
                              кредит на свадьбу                                 NaN   
          очень высокий доход кредит на автомобиль                         8.333333   
                              кредит на недвижимость и жилье              10.810811   
                              кредит на образование                        8.695652   
                              кредит на свадьбу                                 NaN   
          средний доход       кредит на автомобиль                         0.000000   
                              кредит на недвижимость и жилье              14.583333   
                              кредит на образование                        4.000000   
                              кредит на свадьбу                                 NaN   

family_status                                                 в разводе  \
children  group_income        group_purpose                               
без детей высокий доход       кредит на автомобиль             7.317073   
                              кредит на недвижимость и жилье   4.464286   
                              кредит на образ

Исходя из таблицы видно, что при слиянии нескольких факторов образуются наиболее критичные категории заемщиков, те группы у которых более 20% являются, на мой взгляд, проблемныи лицами. К таким гражданам стоит присмотреться. К примеру, у вдовцов с детьми без отраженного дохода или его не имеющих - 100 % задолженность по кредитам на автомобили и образование, хотя в целом вдовцы имеют наибольший процент возвратов как отдельная категория. Возможно не хватает данных выборке в этом случае.
А у разведенных с детьми и доходом до 50 000 р. каждый второй не вернул кредит в срок на автомобиль


Если смотреть отдельно в разрезе какого либо фактора то можно сделать следующие заключения:
- Четкой зависимости между надежностью и наличием детей нет. Однако в целом лица, без детей имеют большее кредитную надежность, чем лица имеющие детей.
- Явную тенеденцию по семейному положению тоже нельзя выделить, отдельно можно отметить, что наибольшая надежность по возвратам у вдов/вдовцов.
- Уровень дохода не сильно влияет на то, как люди возвращают кредит. По лицам с наименьшей доходной группой судить сложно так как их доля в выборке не высока.
- По целям кредитования сильного разброса нет между группами. Наиболее благоприятная группа это кредитование жилья.


## Общий вывод

Отдельные факторы, такие как доход, семейное положение, наличие детей или цели кредита по отдельности при формировании решения о кредите не дают особой информации о платежеспособности заемщика. Построив сводную таблицу по совокупности выделенных в ходе исследования категорий можно выделить ряд групп людей имеющий высокие проценты по наличию задолженностей, в которых чуть ли не каждый пятый, а то и каждый имел такую задолженность. Результат наглядно отражен в сводной таблице в процентах. Данные категории людей необходимо более детально изучить и понять есть ли релаьная законочерность в данных случаях. 